In [ ]:
dataset_path="DATA/TRAIN_1_IRREGULAR_STEPS_V2.json"
dataset_path_2="DATA/TRAIN_2_IRREGULAR_STEPS.json"

In [ ]:
import torch
import numpy as np
from sklearn import metrics


from ManeuverDetectionDataset import ManeuverDetectionDataset, IrregularDataset, SlidingWindowDataset
from torch.utils.data import DataLoader

def get_l_out(l_in, kernel_size, padding=0, dilation=1, stride=1):
    return np.floor((l_in + 2 * padding - dilation * (kernel_size - 1) -1)/stride + 1)

class ConvBlock1d(torch.nn.Module):
    def __init__(self, conv_kwargs, pool_kwargs, dropout_rate) -> None:
        super().__init__()

        # https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html
        self.conv = torch.nn.Conv1d(**conv_kwargs)
        self.activation_fn = torch.nn.ReLU()
        # https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html
        self.pooling = torch.nn.MaxPool1d(**pool_kwargs)
        self.dropout = torch.nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.conv(x)
        x = self.activation_fn(x)
        x = self.pooling(x)
        x = self.dropout(x)
        return x

class Cnn1d(torch.nn.Module):
    def __init__(self, block_kwargs_list, linear_kwargs) -> None: # use_dv_head=False, use_date_head=False
        super().__init__()
        self.block_kwargs_list = block_kwargs_list
        ll = []
        for block_kwargs in block_kwargs_list:
            ll.append(ConvBlock1d(**block_kwargs))

        self.convnet = torch.nn.Sequential(*ll)
        # https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
        self.fcnn = torch.nn.Linear(**linear_kwargs)
        self.activation_fn = torch.nn.ReLU()

        self.classification_head = torch.nn.Sequential(*[
            torch.nn.Linear(linear_kwargs['out_features'], 1),
            # torch.nn.Softmax(dim=-1)
        ])
        
    def forward(self, x):
        # main model
        x = self.convnet(x)
        embedding = torch.flatten(x, start_dim=1) # size : batch size x length
        x = self.fcnn(embedding)
        x = self.activation_fn(x)

        # classification head
        c = self.classification_head(x)
        return torch.squeeze(c), embedding
    
    # def predict(self, x, return_embedding=False):
    #     self.eval()
    #     # main model
    #     x = self.convnet(x)
    #     embedding = torch.flatten(x, start_dim=1) # size : batch size x length
    #     x = self.fcnn(embedding)
    #     x = self.activation_fn(x)

    #     # classification head
    #     c = self.classification_head(x)
    #     if(return_embedding):
    #         return c, embedding
    #     return x


In [ ]:
from tqdm import tqdm
class TorchTrainer:
    def __init__(self, model, verbose=True, weight=None, index_y=0, loss_function="BCELoss") -> None:
        self.model = model
        self.weight = weight
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self.model.to(self.device)
        if(loss_function == 'CrossEntropyLoss'): # DO NOT USE IT
            self.loss_function = torch.nn.CrossEntropyLoss(weight=self.weight) # torch.tensor([0.05, 0.95]))
        elif(loss_function == "L1Loss"):
            self.loss_function = torch.nn.L1Loss()
        elif(loss_function == 'BCELoss'):
            loss_function =  torch.nn.BCEWithLogitsLoss(weight=self.weight)
            self.loss_function = lambda pred, target: loss_function(pred, target.to(torch.float))
        self.optimiser = torch.optim.SGD(self.model.parameters(), lr=1e-3, momentum=0.9)
        self.verbose = verbose
        self.index_y = index_y
    
    def train(self, train_loader, epochs, lr):
        train_loss_list = []
        scheduler = torch.optim.lr_scheduler.CyclicLR(self.optimiser, base_lr=lr/2., max_lr=2 * lr)
        t = tqdm(range(epochs), desc='0 - loss: 0', leave=True, disable=not self.verbose)
        for epoch in t:
            train_loss = self.train_one_epoch(train_loader, scheduler)
            train_loss_list.append(train_loss)
            t.set_description("{} - loss: {:0.2f}".format(epoch, train_loss), refresh=True)
        return train_loss_list

    def train_one_epoch(self, train_loader, scheduler):
        loss_list = []
        for x, y in train_loader:
            self.optimiser.zero_grad()
            x = x.to(self.device)
            y = tuple([y_.to(self.device) for y_ in y])
            pred, embedding = self.model(x)
            loss = self.loss_function(pred, y[self.index_y])
            loss.backward()
            self.optimiser.step()
            loss_list.append(loss.detach().cpu().numpy())
            scheduler.step()

        return np.mean(loss_list)

    def predict(self, test_loader, return_true=False):
        self.model.eval()
        c_pred_list = []
        c_true_list = []
        with torch.no_grad():
            for x, y in test_loader:
                x = x.to(self.device)
                y = tuple([y_.to(self.device) for y_ in y])
                pred = self.model(x)
                c_pred_list.append(pred)
                # dv_pred_list.append(pred[1])
                c_true_list.append(y[self.index_y])
                # dv_true_list.append(y[1])
        self.model.train()

        pred_tuple = (torch.concatenate(c_pred_list, axis=0),)
                # torch.concatenate(dv_pred_list, axis=0))        
        if(return_true):
            true_tuple = (torch.concatenate(c_true_list, axis=0),)
                # torch.concatenate(dv_true_list, axis=0))
            return true_tuple, pred_tuple
        return pred_tuple


# Phase 1 :
Detect which time series contain maneuvers.

CNN to determine on full time series, not evenly spaced, if it contains a maneuver or not.

Fixed size of the time series : 1000 (48h of data).

In [ ]:
block_kwargs_list_1000 = [
    { # layer 1
        'conv_kwargs': {
            'in_channels': 3,
            'out_channels': 4,
            'kernel_size': 7,
            'stride': 1,
            'padding': 0,
            'dilation': 1,
            'groups': 1,
            'bias': True,
            'padding_mode': 'zeros'
        },
        'pool_kwargs': {
            'kernel_size': 7,
            'stride': None,
            'padding': 0,
            'dilation': 1
        },
        'dropout_rate': 0.0
    },
    { # layer 2
        'conv_kwargs': {
            'in_channels': 4,
            'out_channels': 6,
            'kernel_size': 7,
            'stride': 1,
            'padding': 0,
            'dilation': 1,
            'groups': 1,
            'bias': True,
            'padding_mode': 'zeros'
        },
        'pool_kwargs': {
            'kernel_size': 7,
            'stride': None,
            'padding': 0,
            'dilation': 1
        },
        'dropout_rate': 0.0
    },
    { # layer 3
        'conv_kwargs': {
            'in_channels': 6,
            'out_channels':6,
            'kernel_size': 5,
            'stride': 1,
            'padding': 0,
            'dilation': 1,
            'groups': 1,
            'bias': True,
            'padding_mode': 'zeros'
        },
        'pool_kwargs': {
            'kernel_size': 5,
            'stride': None,
            'padding': 0,
            'dilation': 1
        },
        'dropout_rate': 0.0
    }
]
linear_kwargs_1000 = {
    'in_features': 18,
    'out_features': 10 # size of the projection space (dimension reduction)
}

conv_net_1000 = Cnn1d(block_kwargs_list_1000, linear_kwargs_1000).float()

# test
c = conv_net_1000(torch.zeros(4, 3, 1000).float())

## Data

In [22]:

train_dataset = IrregularDataset([
    ManeuverDetectionDataset(dataset_path, dataset_type="TRAIN"),
    ManeuverDetectionDataset(dataset_path_2, dataset_type="TRAIN")])
valid_dataset = IrregularDataset([
    ManeuverDetectionDataset(dataset_path, dataset_type="VALIDATION"),
    ManeuverDetectionDataset(dataset_path_2, dataset_type="VALIDATION")])

train_loader = DataLoader(train_dataset, batch_size=8, drop_last=True, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, drop_last=True, shuffle=False)

## TCNN

In [ ]:
from tcnn import TCNModel
tcn_model = TCNModel(num_channels=[20, 20]).float()

In [ ]:
trainer_tcnn = TorchTrainer(model=tcn_model)

In [ ]:
for lr in [1e-3, 5e-4, 5e-3]: # 5e-3, 2e-3, 
    print(trainer_tcnn.train(train_loader, epochs=7, lr=lr))

In [ ]:
true_valid, pred_valid = trainer_tcnn.predict(valid_loader, return_true=True)
true_train, pred_train = trainer_tcnn.predict(train_loader, return_true=True)

In [ ]:
print(torch.count_nonzero((pred_valid[0] > 0.5) == true_valid[0])/true_valid[0].shape[0])
print(torch.count_nonzero((pred_train[0] > 0.5) == true_train[0])/true_train[0].shape[0])

y_true, y_pred = true_train[0].cpu().numpy(), (pred_train[0]>0.5).cpu().numpy()
print(metrics.confusion_matrix(y_true, y_pred))

y_true, y_pred = true_valid[0].cpu().numpy(), (pred_valid[0]>0.5).cpu().numpy()
print(metrics.confusion_matrix(y_true, y_pred))

## CNN

In [ ]:
conv_net_1000 = Cnn1d(block_kwargs_list_1000, linear_kwargs_1000).float()

In [ ]:
trainer = TorchTrainer(model=conv_net_1000)

In [ ]:
for lr in [1e-3, 5e-4]: # 5e-3, 2e-3,  2e-4, 1e-4, 5e-4
    print(trainer.train(train_loader, epochs=4, lr=lr))

In [ ]:
true_valid, pred_valid = trainer.predict(valid_loader, return_true=True)
true_train, pred_train = trainer.predict(train_loader, return_true=True)

In [ ]:
print(torch.count_nonzero(pred_valid[0] > 0.5 == true_valid[0])/true_valid[0].shape[0])
print(torch.count_nonzero(pred_train[0] > 0.5 == true_train[0])/true_train[0].shape[0])

y_true, y_pred = true_train[0].cpu().numpy(), (pred_train[0]>0.5).cpu().numpy()
print(metrics.confusion_matrix(y_true, y_pred))

y_true, y_pred = true_valid[0].cpu().numpy(), (pred_valid[0]>0.5).cpu().numpy()
print(metrics.confusion_matrix(y_true, y_pred))

# Second Step - First solution

Now that we have the problematric ones, we can try to determine both the dV and the time of the maneuver.
How do we do ?

We use a simple linear model on the embedding.

In [ ]:

train_dataset_man_only = IrregularDataset([
    ManeuverDetectionDataset(dataset_path, dataset_type="TRAIN", filter_samples='MANEUVER_ONLY'),
    ManeuverDetectionDataset(dataset_path_2, dataset_type="TRAIN", filter_samples='MANEUVER_ONLY')])
valid_dataset_man_only = IrregularDataset([
    ManeuverDetectionDataset(dataset_path, dataset_type="VALIDATION", filter_samples='MANEUVER_ONLY'),
    ManeuverDetectionDataset(dataset_path_2, dataset_type="VALIDATION", filter_samples='MANEUVER_ONLY')])

train_loader_man_only = DataLoader(train_dataset_man_only, batch_size=8, drop_last=False, shuffle=True)
valid_loader_man_only = DataLoader(valid_dataset_man_only, batch_size=8, drop_last=False, shuffle=False)

### Heads & Wrapper

In [ ]:
class ManeuverTimeHead(torch.nn.Module):
    def __init__(self, in_features) -> None:
        super().__init__()
        self.linear1 = torch.nn.Linear(in_features, out_features=5)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(5, out_features=1)
        tanh = torch.nn.Tanh()
        self.output_fn = lambda x : 0.5 * (1.0 + tanh(x)) # torch.nn.Sigmoid()

    def forward(self, embedding):
        x = self.linear1(embedding)
        x = self.relu(x)
        x = self.linear2(x)
        return torch.squeeze(self.output_fn(x))

class DeltaVelocityHead(torch.nn.Module):
    def __init__(self, in_features) -> None:
        super().__init__()
        self.linear1 = torch.nn.Linear(in_features, out_features=5)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(5, out_features=1)
        tanh = torch.nn.Tanh()
        self.output_fn = lambda x : 0.5 * (1.0 + tanh(x)) # torch.nn.Sigmoid()

    def forward(self, embedding):
        x = self.linear1(embedding)
        x = self.relu(x)
        x = self.linear2(x)
        return torch.squeeze(self.output_fn(x)) # Dv max is like 1.5 in absolute value
    
class Wrapper(torch.nn.Module):
    def __init__(self, convnet, model_to_train)  -> None:
        super().__init__()
        self.convnet = convnet
        self.model_to_train = model_to_train
    
    def forward(self, x):
        c, embedding = self.convnet(x) # NOTE : we suppose we only send in data with manoeuver in it
        output = self.model_to_train(embedding) 
        return output, embedding

In [ ]:
maneuver_time_net = ManeuverTimeHead(linear_kwargs_1000['in_features'])
dv_net = DeltaVelocityHead(linear_kwargs_1000['in_features'])

# freeze network
for param in conv_net_1000.parameters():
    param.requires_grad = False

time_net_wrapper = Wrapper(conv_net_1000, maneuver_time_net)
dv_net_wrapper = Wrapper(conv_net_1000, dv_net)

In [ ]:
time_trainer = TorchTrainer(model=time_net_wrapper, index_y=2, loss_function='L1Loss')

In [ ]:
for lr in [2e-3, 1e-3, 5e-4]: # 2e-3, 1e-3, 
    print(time_trainer.train(train_loader_man_only, epochs=5, lr=lr))


In [ ]:
dv_trainer = TorchTrainer(model=dv_net_wrapper, index_y=1, loss_function='L1Loss')

In [ ]:
for lr in [2e-3, 1e-3, 5e-4]:
    print(dv_trainer.train(train_loader_man_only, epochs=5, lr=lr))

In [ ]:
true_time, pred_time = time_trainer.predict(valid_loader_man_only, return_true=True)
true_dv, pred_dv = dv_trainer.predict(valid_loader_man_only, return_true=True)

In [ ]:
l1_loss = torch.nn.L1Loss()
print(l1_loss(true_time[0], pred_time[0]))
print(l1_loss(true_dv[0], pred_dv[0]))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots()
sns.scatterplot(true_time[0].cpu().numpy(), pred_time[0].cpu().numpy())

# Re-Training the cnn too ! 

In [ ]:
conv_net_100_time = Cnn1d(block_kwargs_list_1000, linear_kwargs_1000).float()
conv_net_100_dv = Cnn1d(block_kwargs_list_1000, linear_kwargs_1000).float()
conv_net_100_time.load_state_dict(conv_net_1000.state_dict())
conv_net_100_dv.load_state_dict(conv_net_1000.state_dict())
time_net_wrapper_bis = Wrapper(conv_net_100_time, maneuver_time_net)
dv_net_wrapper_bis = Wrapper(conv_net_100_dv, dv_net)

for param in time_net_wrapper_bis.parameters():
    param.requires_grad = True

for param in dv_net_wrapper_bis.parameters():
    param.requires_grad = True

In [ ]:
time_trainer_bis = TorchTrainer(model=time_net_wrapper_bis, index_y=2, loss_function='L1Loss')

In [ ]:
for lr in [2e-3, 1e-3, 5e-4, 2e-4, 1e-4, 5e-5]:
    print(time_trainer_bis.train(train_loader_man_only, epochs=3, lr=lr))


In [ ]:
dv_trainer_bis = TorchTrainer(model=dv_net_wrapper_bis, index_y=1, loss_function='L1Loss')

In [ ]:
for lr in [2e-3, 1e-3, 5e-4, 2e-4, 1e-4, 5e-5]:
    print(dv_trainer_bis.train(train_loader_man_only, epochs=3, lr=lr))

In [ ]:
true_time, pred_time = time_trainer_bis.predict(valid_loader_man_only, return_true=True)
true_dv, pred_dv = dv_trainer_bis.predict(valid_loader_man_only, return_true=True)

In [ ]:
l1_loss = torch.nn.L1Loss()
print(l1_loss(true_time[0], pred_time[0]))
print(l1_loss(true_dv[0], pred_dv[0]))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots()
sns.scatterplot(true_time[0].cpu().numpy(), pred_time[0].cpu().numpy())

## Inference time !

In [ ]:

class InferenceWrapper(torch.nn.Module):
    def __init__(self, convnet, dv_net, time_net) -> None:
        super().__init__()
        self.convnet = convnet
        self.dv_net = dv_net
        self.time_net = time_net
    
    def forward(self, x):
        c, embedding = self.convnet.predict(x)
        dv = self.dv_net(embedding) 
        time = self.time_net(embedding)
        return (c, dv, time), embedding
    
    def predict(self, dataloader, use_convnet_embedding=False):
        self.eval()
        cc_list = []
        time_list = []
        time_true = []
        dv_list = []
        with torch.no_grad():
            for x, y in dataloader:
                c, embedding = self.convnet.predict(x, return_embedding=True)
                # cc = torch.argmax(c, dim=-1)
                # cache = cc == 1
                # dv = torch.zeros(cc.shape)
                # time = torch.zeros(cc.shape)
                if(use_convnet_embedding):
                    dv = self.dv_net(embedding)
                    time = self.time_net(embedding)
                else:
                    dv = self.dv_net(x)
                    time = self.time_net(x)
                cc_list.append(cc.cpu().numpy())
                time_list.append(time.cpu().numpy())
                time_true.append(y[2].cpu().numpy())
                dv_list.append(dv.cpu().numpy())
        return ((np.concatenate(cc_list),
                np.array(dv_list).flatten(),
                np.array(time_list).flatten()), 
                (np.array(time_true)))

In [ ]:
test_dataset_path="DATA/TEST_FILE_PUBLIC.json"
test_dataset= ManeuverDetectionDataset(test_dataset_path, dataset_type="TEST")
test_dataset_irr = IrregularDataset([ManeuverDetectionDataset(dataset_path, dataset_type="TEST", imported_dataset=test_dataset.dataset)])
test_loader = DataLoader(test_dataset_irr, batch_size=1, drop_last=True)

## XGBOOST

In [ ]:
# xgboost
import pickle
file_name = "xgb_reg.pkl"
# load
xgb_model_loaded = pickle.load(open(file_name, "rb"))

with np.load('interpolated_ra_test.npz') as data:
    x_scaled = data['x_scaled']
    test_is_maneuver = data['is_maneuver']
    test_feature_ra = data['feature_ra']
    test_x_scaler = data['x_scaler']
    test_y_scalers = data['y_scalers']

In [ ]:
pred_classification = xgb_model_loaded.predict(test_feature_ra)

## INFERENCE WRAPPER

In [ ]:
# freeze network

inference_wrapper = InferenceWrapper(
        convnet=conv_net_1000, # which one ???
        dv_net=dv_net_wrapper_bis,
        time_net=time_net_wrapper_bis
)
for param in inference_wrapper.parameters():
    param.requires_grad = False

In [ ]:
preds, true = inference_wrapper.predict(test_loader)

In [ ]:
pred = np.stack(preds, axis=1) 
# print(pred.shape)

In [ ]:
pred[:,2]=48*3600*pred[:, 2]

In [ ]:
pred[:, 1] = pred_classification

In [ ]:
from SubmissionGenerator import create_submission
import numpy as np
# pred[:,1]=0.01*np.ones((len(test_dataset))) #dv
create_submission(pred,"DATA/vg1")

# Second step - Second Solution : for those we detected an anomaly for
We split them in several subseries (sliding windows) and we have to find where the anomaly start occuring. 
For each small window, we determine a new score of anomaly. We can refine as many time as required.

May be now we can try predicting the time of the anomaly occuring. I am not sure.

In [ ]:
from ManeuverDetectionDataset import ManeuverDetectionDataset, ManeuverDetectionSlidingWindowDataset
from torch.utils.data import DataLoader

evenly_spaced_dataset_path="DATA/TRAIN_1_EVENLY_SPACED_V2.json"
evenly_spaced_dataset= ManeuverDetectionDataset(evenly_spaced_dataset_path, fixed_step=True) # window_size=30)
evenly_spaced_dataset_sliding_window = ManeuverDetectionSlidingWindowDataset(evenly_spaced_dataset, window_size=433)
# feature,is_maneuver,maneuver_dv,maneuver_time =next(iter(evenly_spaced_loader))
# print(f"features shape (batch size * nb of meas * nb of feature):{feature.shape}\nis maneuver: {is_maneuver.item()}\ndv (m/s): {maneuver_dv.item()}\nmaneuver date (seconds from the observation start): {maneuver_time.item()}")

In [ ]:
evenly_spaced_loader = DataLoader(evenly_spaced_dataset_sliding_window, batch_size=32, drop_last=True, shuffle=True)

In [ ]:
trainer.train(evenly_spaced_loader, epochs=3)

In [ ]:
evenly_spaced_dataset_valid = ManeuverDetectionDataset(evenly_spaced_dataset_path, dataset_type='VALIDATION', fixed_step=True)
valid_set = ManeuverDetectionSlidingWindowDataset(evenly_spaced_dataset_valid, window_size=30)

In [ ]:
valid_loader = DataLoader(evenly_spaced_dataset_sliding_window, batch_size=1, drop_last=False, shuffle=False)

In [ ]:
true, pred = trainer.predict(valid_loader, return_true=True)

In [ ]:
torch.count_nonzero((pred[0] > 0.5) == true[0])/true[0].shape[0]

In [ ]:
from sklearn import metrics
y_true, y_pred = true[0].cpu().numpy(), torch.argmax(pred[0], dim=1).cpu().numpy()
metrics.confusion_matrix(y_true, y_pred) # problem with those definitely - class are completly UNBALANCED. 

In [ ]:
block_kwargs_list_30 = [
    { # layer 1
        'conv_kwargs': {
            'in_channels': 2,
            'out_channels': 4,
            'kernel_size': 3,
            'stride': 1,
            'padding': 0,
            'dilation': 1,
            'groups': 1,
            'bias': True,
            'padding_mode': 'zeros'
        },
        'pool_kwargs': {
            'kernel_size': 3,
            'stride': None,
            'padding': 0,
            'dilation': 1
        },
        'dropout_rate': 0
    },
    { # layer 2
        'conv_kwargs': {
            'in_channels': 4,
            'out_channels': 4,
            'kernel_size': 3,
            'stride': 1,
            'padding': 0,
            'dilation': 1,
            'groups': 1,
            'bias': True,
            'padding_mode': 'zeros'
        },
        'pool_kwargs': {
            'kernel_size': 3,
            'stride': None,
            'padding': 0,
            'dilation': 1
        },
        'dropout_rate': 0
    }
]
linear_kwargs_30 = {
    'in_features': 8,
    'out_features': 10 # size of the projection space (dimension reduction)
}
conv_net_30 = Cnn1d(block_kwargs_list_30, linear_kwargs_30).float()


In [ ]:
19916/536388 * 100 # 3.72% 

In [ ]:
cross_entropy_loss = torch.nn.CrossEntropyLoss() # weight=torch.tensor([0.05, 0.95]))
mae_loss_1 = torch.nn.L1Loss()
mae_loss_2 = torch.nn.L1Loss()
def total_loss_function(pred, true, alpha=0.5, beta=0.1, gamma=0.4):
    c_true, dv_true, date_true = true
    c, dv, date = pred
    return alpha * cross_entropy_loss(c, c_true) + beta * mae_loss_1(dv, dv_true) + gamma * mae_loss_2(date_true, date)
  